# TimeRNN class
* ./common/time_layers.py

In [ ]:
class TimeRNN: # 시간 블럭 크기만큼 RNN instance를 옆으로 이어붙인 클래스
    def __init__(self, Wx, Wh, b, stateful=False): # stateful 옵션이 rnn instance와 차이점. 맨 마지막 히든스테이트를 다음으로 보내느냐(True), 아니냐 (False)
        self.params = [Wx, Wh, b]
        self.grads = [np.zeros_like(Wx), np.zeros_like(Wh), np.zeros_like(b)] # 제로벡터 생성 rnn instance와 동일
        self.layers = None

        self.h, self.dh = None, None
        self.stateful = stateful

    def forward(self, xs):
        Wx, Wh, b = self.params
        N, T, D = xs.shape # 3개 N: 배치사이즈 길이(크기), T: 시간 블럭 크기, D: Dense vector의 차원
        D, H = Wx.shape # Wx: 들어오는 데이터 Dense vector, H: 은닉층(hidden_state로 들어가는 vec이든 나오는 vec이든 크기 동일.)

        self.layers = []
        hs = np.empty((N, T, H), dtype='f') # hs: 히든 스테이트들의 모임 (각 시간대별로 히든스테이트가 모일 것. 복수형이라 변수에 s붙임.)
                # 왜 np.empty인가? 빈행렬에 아무거나 넣어주는... (일단 아무거나 채워놓고 나중에 제대로된 값 삽입.)

        if not self.stateful or self.h is None:
            self.h = np.zeros((N, H), dtype='f') # stateful=F면 다음시간으로 전달안해 == 제로행렬 전달하면 전달안하는 것과 같튼 말

        for t in range(T):
            layer = RNN(*self.params) # instance 또 새로 생성
            self.h = layer.forward(xs[:, t, :], self.h) # 방금 출력한 걸로 맨 왼쪽 열을 바꾸겠다.
            hs[:, t, :] = self.h
            self.layers.append(layer) # layer = rnn instance

        return hs

    def backward(self, dhs):
        Wx, Wh, b = self.params
        N, T, H = dhs.shape
        D, H = Wx.shape

        dxs = np.empty((N, T, D), dtype='f')
        dh = 0
        grads = [0, 0, 0]
        for t in reversed(range(T)):
            layer = self.layers[t]
            dx, dh = layer.backward(dhs[:, t, :] + dh)
            dxs[:, t, :] = dx

            for i, grad in enumerate(layer.grads):
                grads[i] += grad

        for i, grad in enumerate(grads):
            self.grads[i][...] = grad
        self.dh = dh

        return dxs

    def set_state(self, h):
        self.h = h

    def reset_state(self):
        self.h = None